In [ ]:
import requests
from google.colab import drive
import glob
import pandas as pd

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
import time

In [ ]:
import json
import urllib.request
import urllib.parse

jobs=[]

'''
See https://www.openfigi.com/api for more information.
'''

openfigi_apikey = 'e35acc15-9da6-458d-8f7f-2ef3b0b602ea'  # Put API Key here

def map_jobs(jobs):
    '''
    Send an collection of mapping jobs to the API in order to obtain the
    associated FIGI(s).
    Parameters
    ----------
    jobs : list(dict)
        A list of dicts that conform to the OpenFIGI API request structure. See
        https://www.openfigi.com/api#request-format for more information. Note
        rate-limiting requirements when considering length of `jobs`.
    Returns
    -------
    list(dict)
        One dict per item in `jobs` list that conform to the OpenFIGI API
        response structure.  See https://www.openfigi.com/api#response-fomats
        for more information.
    '''
    handler = urllib.request.HTTPHandler()
    opener = urllib.request.build_opener(handler)
    openfigi_url = 'https://api.openfigi.com/v2/mapping'
    request = urllib.request.Request(openfigi_url, data=bytes(json.dumps(jobs), encoding='utf-8'))
    request.add_header('Content-Type','application/json')
    if openfigi_apikey:
        request.add_header('X-OPENFIGI-APIKEY', openfigi_apikey)
    request.get_method = lambda: 'POST'
    connection = opener.open(request)
    if connection.code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return json.loads(connection.read().decode('utf-8'))


def job_results_handler(jobs, job_results):
    '''
    Handle the `map_jobs` results.  See `map_jobs` definition for more info.
    Parameters
    ----------
    jobs : list(dict)
        The original list of mapping jobs to perform.
    job_results : list(dict)
        The results of the mapping job.
    Returns
    -------
        None
    '''
    for job, result in zip(jobs, job_results):
        job_str = '|'.join(job.values())
        figis_str = result.get('data', [])
        result_str = figis_str or result.get('error')
        return figis_str


def main(jobs = jobs):
    '''
    Map the defined `jobs` and handle the results.
    Returns
    -------
        None
    '''
    job_results = map_jobs(jobs)
    output_main = job_results_handler(jobs, job_results)
    return output_main


In [ ]:
df = pd.read_csv("/content/drive/My Drive/Analytics in Practice APG/Data/data_all.csv")
df['TICKER'] = 'NaN'
df

,Manager Name,NAME OF ISSUER,TITLE OF CLASS,CUSIP,VALUE (x$1000),INVESTMENT DISCRETION,OTHER MANAGER,SHRS OR PRN AMT,SH/PRN,VOTING AUTHORITY SOLE,VOTING AUTHORITY SHARED,VOTING AUTHORITY NONE,NEW_CUSIP,sd,CIK,PUT/CALL,TICKER
0,"Bridgewater Associates, LP",ACCENTURE PLC IRELAND,SHS Class A,G1151C101,317,SOLE,NaN,4400,SH,4400,0,0,#G1151C101,10,1350694,NaN,NaN
1,"Bridgewater Associates, LP",ACE LTD,SHS,H0023R105,652,SOLE,NaN,7285,SH,7285,0,0,#H0023R105,10,1350694,NaN,NaN
2,"Bridgewater Associates, LP",AES CORP,COM,00130H105,546,SOLE,NaN,45500,SH,45500,0,0,#00130H105,10,1350694,NaN,NaN
3,"Bridgewater Associates, LP",AETNA INC NEW,COM,00817Y108,1943,SOLE,NaN,30577,SH,30577,0,0,#00817Y108,10,1350694,NaN,NaN
4,"Bridgewater Associates, LP",AFFILIATED MANAGERS GROUP,COM,8252108,617,SOLE,NaN,3762,SH,3762,0,0,#008252108,10,1350694,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30603,Third Point LLC,TIBCO SOFTWARE INC,COM,88632Q103,107025,SOLE,NaN,5000000,SH,5000000,0,0,#88632Q103,10,1040273,NaN,NaN
30604,Third Point LLC,TIFFANY amp; CO NEW,COM,886547108,120186,SOLE,NaN,1650000,SH,1650000,0,0,#886547108,10,1040273,NaN,NaN
30605,Third Point LLC,WESCO INTL INC,COM,95082P105,74756,SOLE,NaN,1100000,SH,1100000,0,0,#95082P105,10,1040273,NaN,NaN
30606,Third Point LLC,WILLIAMS COS INC DEL,COM,969457100,56822,SOLE,NaN,1750000,SH,1750000,0,0,#969457100,10,1040273,NaN,NaN


In [ ]:
list1 = df['NEW_CUSIP']
list2 = list(set(list1))
new_list = []
for i in list2:
  print((str(i)))
  new_list.append(i[1:])


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
#759509102
#553829102
#56035L104
#15132H101
#29273R109
#19259PAF9
#292475209
#14055X102
#709102107
#29285W104
#N63218106
#05591B109
#404303109
#48273U102
#19259P300
#517834107
#511637100
#91359E105
#494368103
#930427109
#42722X106
#57060U191
#553810102
#00206R102
#118255108
#948596101
#868536103
#00737L103
#761283100
#P31076105
#88165N105
#210502100
#65336K103
#92276F100
#631158102
#192108504
#16208T102
#461203AD3
#883211104
#590876306
#464286908
#00207R101
#156700106
#01748H107
#559080106
#02215L100
#723456109
#257867951
#46429B598
#74267C106
#21976U109
#25065K104
#57722W106
#756207106
#85571BAA3
#116794108
#G5138L100
#750438953
#909839102
#74973WAB3
#014491104
#671044105
#737630103
#75040P405
#89628EAA2
#05580M108
#680033107
#09857L108
#05350V106
#678026105
#594918104
#Y26496201
#252603105
#90400D108
#97651M109
#315405100
#70086Y105
#20368X101
#G36738105
#92840M102
#02153X108
#584688AG0
#G16249107
#500472303
#023135106
#81369Y906
#866942105
#G0464B

In [ ]:
len(new_list)

5628

In [ ]:
list2 = new_list
data2 = pd.DataFrame()
data2['CUSIP'] = list2
data2['DATA'] = 'NaN'
print(data2)

for index, row in data2.iterrows():
  if data2.loc[index, 'DATA'] == 'NaN':
    try:
      all_result = main(jobs = [{"idType":"ID_CUSIP","idValue":"{}".format(data2.loc[index,'CUSIP'])}])
      data2.loc[index, 'RAW'] = str(all_result)
      data2.loc[index, 'DATA'] = str(all_result[0])
      data2.loc[index, 'figi'] = eval(data2.loc[index, 'DATA']).get('figi')
      data2.loc[index, 'name'] = eval(data2.loc[index, 'DATA']).get('name')
      data2.loc[index, 'ticker'] = eval(data2.loc[index, 'DATA']).get('ticker')
      data2.loc[index, 'exchCode'] = eval(data2.loc[index, 'DATA']).get('exchCode')
      data2.loc[index, 'compositeFIGI'] = eval(data2.loc[index, 'DATA']).get('compositeFIGI')
      data2.loc[index, 'uniqueID'] = eval(data2.loc[index, 'DATA']).get('uniqueID')
      data2.loc[index, 'securityType'] = eval(data2.loc[index, 'DATA']).get('securityType')
      data2.loc[index, 'marketSector'] = eval(data2.loc[index, 'DATA']).get('marketSector')
      data2.loc[index, 'shareClassFIGI'] = eval(data2.loc[index, 'DATA']).get('shareClassFIGI')
      data2.loc[index, 'uniqueIDFutOpt'] = eval(data2.loc[index, 'DATA']).get('uniqueIDFutOpt')
      data2.loc[index, 'securityType2'] = eval(data2.loc[index, 'DATA']).get('securityType2')
      data2.loc[index, 'securityDescription'] = eval(data2.loc[index, 'DATA']).get('securityDescription')
      print(data2.loc[index])
    except Exception as e:
      print('Error' + data2.loc[index, 'CUSIP'])      
      pass
    #print(data2.loc[index, 'CUSIP'] + '  ' +data2.loc[index, 'TICKER'])
    data2.to_csv('/content/drive/My Drive/Analytics in Practice APG/Data/With_Ticker_all_final.csv')  
    time.sleep(0.25)

          CUSIP DATA
0     744573106  NaN
1     038923108  NaN
2     608753AA7  NaN
3     002474104  NaN
4     42234Q102  NaN
...         ...  ...
5623  037604105  NaN
5624  524707304  NaN
5625  436893200  NaN
5626  46120E602  NaN
5627  44925C103  NaN

[5628 rows x 2 columns]
CUSIP                                                          744573106
DATA                   {'figi': 'BBG000BQZMH4', 'name': 'PUBLIC SERVI...
RAW                    [{'figi': 'BBG000BQZMH4', 'name': 'PUBLIC SERV...
figi                                                        BBG000BQZMH4
name                                        PUBLIC SERVICE ENTERPRISE GP
ticker                                                               PEG
exchCode                                                              US
compositeFIGI                                               BBG000BQZMH4
uniqueID                                                             NaN
securityType                                                Common

KeyboardInterrupt: ignored

In [ ]:
data2.head(50)

,CUSIP,TICKER
0,759351604,RGA
1,00938A104,AIRG
2,74727D306,QADA
3,19249H103,CHRS
4,01535P106,AXU
5,56117J100,MBUU
6,45841N107,IBKR
7,09251A104,BDJ
8,01741R102,ATD
9,30158L118,ENPC/WS


In [ ]:
import time
df_ticker = pd.read_csv("/content/drive/My Drive/Analytics in Practice APG/Data/With_Ticker_all_final.csv")

for index, row in df_ticker.iterrows():
  if df_ticker.loc[index, 'TICKER'] == 'No identifier found.':
    df_ticker.loc[index, 'TICKER'] = main(jobs = [{"idType":"ID_CINS","idValue":"{}".format(df_ticker.loc[index,'CUSIP'])}])
    print(index)
    df_ticker.to_csv('/content/drive/My Drive/Analytics in Practice APG/Data/With_Ticker_all_final2.csv')  
    time.sleep(0.25)

12
13
14
17
18
33
39
40
41
52
56
75
86
87
92
96
120
129
136
137
140
142
149
158
163
172
175
179
184
191
198
208
209
220
224
230
232
242
244
246
251
256
267
276
290
293
296
297
301
302
303
311
312
314
322
324
329
331
334
335
341
362
367
375
384
387
389
390
394
399
403
419
421
429
433
440
460
463
464
466
471
474
476
477
487
497
500
501
520
526
529
537
542
549
553
560
566
577
578
593
605
606
608
610
612
626
633
658
659
661
672
678
689
692
698
708
710
715
719
741
752
759
762
764
769
773
774
790
794
800
804
813
830
833
834
840
852
855
861
875
877
883
889
892
900
904
906
912
917
933
937
939
943
946
950
956
959
960
967
970
979
985
987
995
996
1006
1012
1017
1027
1029
1033
1037
1044
1059
1065
1070
1081
1085
1109
1114
1119
1128
1151
1154
1156
1175
1176
1178
1195
1197
1207
1214
1222
1228
1238
1253
1259
1267
1269
1272
1282
1292
1294
1297
1298
1302
1303
1310
1329
1331
1351
1352
1363
1364
1379
1384
1389
1394
1399
1407
1408
1417
1435
1450
1453
1455
1461
1464
1488
1490
1518
1521
1526
1530
1548
1549
1

In [ ]:
df_ticker2 = pd.read_csv("/content/drive/My Drive/Analytics in Practice APG/Data/With_Ticker_all3.csv")

for index, row in df_ticker2.iterrows():
  if data2.loc[index, 'TICKER'] == 'Invalid idValue format':
    data2.loc[index, 'TICKER'] = main(jobs = [{"idType":"ID_CUSIP_8_CHR","idValue":"{}".format(data2.loc[index,'CUSIP'])}])
    print(index)
    data2.to_csv('/content/drive/My Drive/Analytics in Practice APG/Data/With_Ticker_all4.csv')  
    time.sleep(0.25)

36
113
120
146
167
175
180
196
224
271
301
318
338
350
372
378
381
420
430
450
460
465
468
470
476
499
501
511
550
551
557
576
584
591
628
650
658
664
673
689
783
789
819
881
891
904
905
920
923
986
1021
1024
1057
1083
1084
1099
1156
1161
1172
1198
1249
1250
1251
1263
1269
1313
1338
1346
1347
1355
1379
1405
1412
1508
1516
1555
1620
1626
1633
1656
1667
1703
1731
1775
1794
1809
1832
1845
1900
1940
1941
1951
1970
2003
2008
2071
2107
2113
2153
2207
2216
2274
2346
2378
2387
2397
2462
2505
2510
2536
2583
2597
2617
2653
2672
2772
2787
2845
2852
2904
2908
2914
2944
2949
2973
2979
2994
3002
3032
3042
3107
3135
3141
3171
3189
3203
3233
3236
3247
3284
3357
3362
3373
3381
3406
3442
3446
3494
3518
3541
3587
3593
3675
3679
3680
3685
3799
3822
3838
3868
3871
3883
3921
3924
3985
4003
4047
4057
4097
4167
4170
4182
4190
4241
4301
4305
4307
4325
4348
4349
4444
4447
4467
4513
4529
4568
4582
4594
4599
4608
4618
4642
4657
4723
4749
4806
4812
4824
4842
4845
4959
4969
4985
5012
5026
5112
5246
5247
5257
5288
5